# Demo Data Warehouse
This notebook will guide you through creating a demo data warehouse of a beverage selling company.
The data warehouse has the following dimensions:
* ⏰ Date
* 🌍 Country of Sales
* 🏬 Sales Channel
* 🍷 Product

Source: [GitHub SQL Tools](https://github.com/schenkfab/sqltools)

## Database Creation

Creates the database, if it doesn't exist yet.
By default, the database will be named Partitioning with the file locations:
* **Data**: F:\DATA\*
* **Log**: G:\Log\*

In [ ]:
IF NOT EXISTS(SELECT 1 FROM sys.databases WHERE name = 'Partitioning')
BEGIN
   CREATE DATABASE [Partitioning]
    CONTAINMENT = NONE
    ON  PRIMARY 
   ( NAME = N'Partitioning', 
   FILENAME = N'E:\DATA\Partitioning.mdf' , SIZE = 1GB , FILEGROWTH = 1GB )
    LOG ON 
   ( NAME = N'Partitioning_log', 
   FILENAME = N'G:\Log\Partitioning_log.ldf' , SIZE = 512MB , FILEGROWTH = 512MB )
END

GO
USE [Partitioning]
GO

In the following sections, were are going to use the RAND() function to generate fact table entries and their corresponding constraints.

If you require a value between `n` and `m` you need to specify the query like this:

`
SELECT n + CAST(RAND() * (m-n+1) AS INT)
`

Following a few example:


In [ ]:
SELECT 1 + CAST(RAND() * 2 AS INT) -- Between 1 and 2

SELECT 1 + CAST(RAND() * 10 AS INT) -- Between 1 and 10

SELECT 5 + CAST(RAND() * 10 AS INT) -- Between 5 and 14 (RAND() * 10 AS INT --> Nr between 0 and 9) + 5 --> Nr between 5 and 14

## Dimensions
During the following statements, the dimensions will be created and propacated with data.

### DimDate
The first dimension, we are creating is the date dimension with some attributes. 

``Note that there is a helper key column DateKey as an identity for our random function to work later on.``

In [ ]:
-- Dimension Date

CREATE TABLE dbo.DimDate (
   [DateID] INT PRIMARY KEY,
   [Date] DATE NOT NULL,
   [Month] INT NOT NULL,
   [Year] INT NOT NULL,
   [Day] INT NOT NULL,
   [DateKey] INT IDENTITY(1,1) NOT NULL
)

GO

After the creation of the dimension table, we are going to add records to the table as well. To do so, we are going to use a recursive CTE giving us all dates between 2019-01-01 and 2019-04-01.

In [ ]:
-- Provision Dates in Range: January - March 2019

TRUNCATE TABLE dbo.DimDate
GO
WITH sample AS (
  SELECT CAST('2019-01-01' AS DATE) AS dt
  UNION ALL
  SELECT DATEADD(dd, 1, dt)
    FROM sample s
   WHERE DATEADD(dd, 1, dt) < CAST('2019-04-01' AS DATE))
INSERT INTO dbo.DimDate
SELECT convert(varchar(8), dt, 112), dt, MONTH(dt), YEAR(dt), DAY(dt)
  FROM sample

### DimCountry
The second dimension we are creating is Country. It defines in which country our products were sold. By default, it creates members:
* Switzerland
* Austria
* Germany
* Iceland

In [ ]:
-- Dimension Country

CREATE TABLE dbo.DimCountry (
    [Id] INT IDENTITY(1,1) NOT NULL,
    [Name] VARCHAR(20) NOT NULL
)
GO

INSERT INTO dbo.DimCountry
VALUES
    ('Switzerland'),
    ('Austria'),
    ('Germany'),
    ('Iceland')

### DimProduct
The products that were sold including a hierarchy. Category --> Name
For the product categories, we are working with different kinds of alcohol:
* Gin
* Whisky
* Rum

In [ ]:
-- Dimension Product

CREATE TABLE dbo.DimProduct (
    [Id] INT IDENTITY(1,1) NOT NULL,
    [Name] VARCHAR(50) NOT NULL,
    [Category] VARCHAR(50) NOT NULL,
    [Price] DECIMAL(14,2) NOT NULL
)
GO

INSERT INTO dbo.DimProduct
VALUES
    ('Hendrick', 'Gin', 26.90),
    ('Monkey 47', 'Gin', 29.90),
    ('Tanqueray', 'Gin', 14.90),
    ('The Illusionist', 'Gin', 29.90),
    ('Gin Mare', 'Gin', 28.90),
    ('Bombay Sapphire', 'Gin', 15.90),
    ('Jack Daniels', 'Whisky', 16.50),
    ('Lagavulin 16 Years Old', 'Whisky', 53.50),
    ('lendronach 12 Years Old', 'Whisky', 32.50),
    ('Aberfeldy 12 Years Old', 'Whisky', 31.50),
    ('Glenfiddich 12 Years Old', 'Whisky', 24.90),
    ('Botucal Reserva Exclusiva', 'Rum', 30.90),
    ('Don Papa', 'Rum', 29.50),
    ('Plantation Barbados 20th Anniversary', 'Rum', 37.50),
    ('Ron Zacapa Centenario 23', 'Rum', 44.90),
    ('Kraken Black Spiced', 'Rum', 16.50)


In [ ]:
-- Dimension SalesChannel

CREATE TABLE dbo.DimSalesChannel (
    [Id] INT IDENTITY(1,1) NOT NULL,
    [Name] VARCHAR(50) NOT NULL
)

GO

INSERT INTO dbo.DimSalesChannel
VALUES
    ('Online Shop'),('Store'),('Social Media')

In [ ]:
-- Fact Sales
USE Partitioning


CREATE TABLE dbo.FactSales (
    [DateId] INT NOT NULL,
    [CountryId] INT NOT NULL,
    [ProductId] INT NOT NULL,
    [SalesChannelId] INT NOT NULL,
    [Quantity] INT NOT NULL,
    [SalesAmount] DECIMAL(16,2) NULL
)

*The following statement will create 1 million record. Depending on your environment, it might be faster to have it being executed in several SSMS windows at once to increase parallelim. Test it* 😎

**Important: I would recommend copy / pasting this into SSMS and execute it there, as Azure Data Studio might lag out with higher numbers of executions.**

In [ ]:
-- Insert Facts:

-- 
USE [Partitioning]
GO

SET NOCOUNT ON
GO

BEGIN TRAN
GO


DECLARE @DateKey INT
DECLARE @DateId INT
DECLARE @CountryId INT
DECLARE @ProductId INT
DECLARE @SalesChannelId INT
DECLARE @Quantity INT
DECLARE @QuantityHelper INT

SELECT @DateKey = (1 + CAST(RAND() * 90 AS INT))
     , @CountryId = (1 + CAST(RAND() * 4 AS INT))
     , @ProductId = (1 + CAST(RAND() * 16 AS INT))
     , @SalesChannelId = (1 + CAST(RAND() * 3 AS INT))
     , @QuantityHelper = (1 + CAST(RAND() * 100 AS INT))

SELECT @Quantity = CASE WHEN @QuantityHelper % 15 = 0 THEN 1
                        WHEN @QuantityHelper % 15 = 1 THEN 1
                        WHEN @QuantityHelper % 15 = 2 THEN 1
                        WHEN @QuantityHelper % 15 = 3 THEN 2
                        WHEN @QuantityHelper % 15 = 4 THEN 2
                        WHEN @QuantityHelper % 15 = 5 THEN 2
                        WHEN @QuantityHelper % 15 = 6 THEN 3
                        WHEN @QuantityHelper % 15 = 7 THEN 3
                        WHEN @QuantityHelper % 15 = 8 THEN 4
                        WHEN @QuantityHelper % 15 = 9 THEN 4
                        WHEN @QuantityHelper % 15 = 10 THEN 5
                        WHEN @QuantityHelper % 15 = 11 THEN 5
                        WHEN @QuantityHelper % 15 = 12 THEN 6
                        WHEN @QuantityHelper % 15 = 13 THEN 7
                        WHEN @QuantityHelper % 15 = 14 THEN 8
                    END

SELECT @DateId = DateId
  FROM dbo.DimDate d
 WHERE d.DateKey = @DateKey

INSERT INTO dbo.FactSales (DateId, CountryId, ProductId, SalesChannelId, Quantity)
VALUES (@DateId, @CountryId, @ProductId, @SalesChannelId, @Quantity)


GO 10000

COMMIT


In [ ]:
-- Update SalesAmount for all newly added facts

UPDATE f
   SET SalesAmount = f.Quantity * p.Price
  FROM dbo.FactSales f
 INNER JOIN dbo.DimProduct p ON p.Id = f.ProductId
 WHERE SalesAmount IS NULL

If you wonder how many records you are loading per second, execute the statement below. 👨‍💻

In [ ]:
-- Speed of inserts


DECLARE @startCount INT
DECLARE @endCount INT


SELECT @startCount = COUNT(*) FROM dbo.FactSales WITH (NOLOCK)

WAITFOR DELAY '00:00:10';

SELECT @endCount = COUNT(*) FROM dbo.FactSales WITH (NOLOCK)

SELECT (1.0 / 10) * (@endCount - @startCount) As [Perf per Second]